In [1]:
import torch
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import matplotlib as mpl
from copy import deepcopy
from tabulate import tabulate

# Download data from WANDB
And put them in dataframes

In [2]:
from utils import obtain_wandb_data

In [3]:
runs_list, runs_list_GAN, runs_list_SRs = obtain_wandb_data(["EMNIST"])

In [4]:
runs_list = runs_list[0]

In [5]:
runs_list=runs_list.sort_values("name")

We have trained with the SR methods using both 3 and 10 simulations for each training step; we will report below the results obtained with 10 samples.

Separate now the runs which use 3 and 10 n_generations (but we include GAN in both)

In [6]:
df_3 = runs_list[(runs_list["num_simulations_generator"]==3)  | (runs_list["loss"] == "cross_entropy")]
df_10 = runs_list[(runs_list["num_simulations_generator"]==10) | (runs_list["loss"] == "cross_entropy")]

The first run did not compute the RMSE values correctly. We therefore computed them subsequently; add them to the dataframe right now: 

In [7]:
names = ["camera_model_kernel_score_10", "camera_model_kernel_score_10_patched_5_8", "camera_model_energy_score_10_patched_5_8",
         "camera_model_energy_score_10_patched_7_14", "camera_model_energy_score_10", "camera_model_kernel_score_10_patched_7_14",
         "camera_model_GAN_800000"]
dicts = [{'rmse_val_mean': 0.06406807, 'rmse_val_std': 0.045899235},
         {'rmse_val_mean': 0.06944673, 'rmse_val_std': 0.04826797},
         {'rmse_val_mean': 0.06270825, 'rmse_val_std': 0.04553171},
         {'rmse_val_mean': 0.07040126, 'rmse_val_std': 0.04799262},
         {'rmse_val_mean': 0.0782916, 'rmse_val_std': 0.0482274},
         {'rmse_val_mean': 0.09903354, 'rmse_val_std': 0.058070697},
         {'rmse_val_mean': 0.25184682, 'rmse_val_std': 0.19277969}]

# loop over names and dicts
for name, dict in zip(names, dicts):
    row_index_1 = df_10["name"] == name 
    row_index_2 = df_3["name"] == name 
    # loop over dict keys
    for key in dict.keys():
        df_10.loc[row_index_1, key] = dict[key]
        df_3.loc[row_index_2, key] = dict[key]        

In [8]:
df_10

,name,ID,epochs,patch_size,patch_step,patched_sr,sample_seed,gen_opt_args,scoring_rule,hold_out_perc,...,dis_opt_args,max_norm_dis,max_norm_gen,num_simulations,dis_grad,dreal_std,dfake_mean,dfake_std,dreal_mean,dis_loss
12,camera_model_GAN_800000,u25w8avb,10000,NaN,NaN,NaN,42,"[0.0002, [0.5, 0.99]]",NaN,NaN,...,"[0.0002, [0.5, 0.99]]",0.01,0.01,800000.0,20.277802,0.025258,0.506794,0.0235,0.496308,1.409826
5,camera_model_energy_score_10,1tka5tp1,10000,NaN,NaN,False,42,"[0.0002, [0.5, 0.99]]",energy_score,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,camera_model_energy_score_10_patched_5_8,11wxayas,10000,8.0,5.0,True,42,"[0.0002, [0.5, 0.99]]",energy_score,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,camera_model_energy_score_10_patched_7_14,1q7dpoiq,10000,14.0,7.0,True,42,"[0.0002, [0.5, 0.99]]",energy_score,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,camera_model_kernel_score_10,v097tq54,10000,NaN,NaN,False,42,"[0.0002, [0.5, 0.99]]",kernel_score,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,camera_model_kernel_score_10_patched_5_8,2bb5fc4j,10000,8.0,5.0,True,42,"[0.0002, [0.5, 0.99]]",kernel_score,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,camera_model_kernel_score_10_patched_7_14,1g7nsdur,10000,14.0,7.0,True,42,"[0.0002, [0.5, 0.99]]",kernel_score,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now print that as a Table: need to first convert the dataframe to a dict of dicts and then print

In [9]:
def print_table(df, latex=False, file=None, tranpose=False):

    metrics = [['rmse_val_std', 'rmse_val_mean'], ['cal_err_val_std', 'cal_err_val_mean'],
                   ['r2_val_std', 'r2_val_mean'], ['_runtime'], ['global_step']]
    #metric_names = ["rmse", "cal err", "r2", "runtime", "early stopping step"]
    metric_names = [r"RMSE $ \downarrow $", r"Cal. Err. $ \downarrow $", r"R$^2$ $ \uparrow $", "Runtime (sec)", "Early stopping epoch"]

    df_dict = df.to_dict("list")
    
    name_list = [""] + [name.replace("camera_model_", "").replace("_", " ").replace("score", "") for name in df_dict["name"]]
    
    all_lists = []
    all_lists.append(name_list)

    for name, metric in zip(metric_names, metrics): 
        inner_list = [name]    

        if len(metric) == 2:
            inner_list += [f"{df_dict[metric[1]][i]:.2f} $\pm$ {df_dict[metric[0]][i]:.2f}" for i in range(len(df_dict[metric[0]]))]
        else:
            inner_list += [f"{df_dict[metric[0]][i]:.2f}" for i in range(len(df_dict[metric[0]]))]
                
        all_lists.append(deepcopy(inner_list))
    
    if tranpose: 
        all_lists = list(map(list, zip(*all_lists)))

    if file is None:
        if latex: 
            print(tabulate(all_lists, headers="firstrow", tablefmt="latex_raw"))
        else:
            print(tabulate(all_lists, headers="firstrow"))
    else:
        with open(file, 'w') as f:
            if latex: 
                f.write(tabulate(all_lists, headers="firstrow", tablefmt="latex_raw"))
            else:
                f.write(tabulate(all_lists, headers="firstrow"))


In [10]:
print_table(df_10, file=f"tables/camera_model_all_metrics_n_gen={10}", latex=True, tranpose=True)

Print a Table with only the best performing method:

In [11]:
df_10_selected = df_10[df_10["name"].isin(["camera_model_GAN_800000", "camera_model_energy_score_10_patched_5_8", "camera_model_kernel_score_10_patched_5_8"])]
print_table(df_10_selected, file=f"tables/camera_model_best_methods_all_metrics_n_gen={10}", latex=True, tranpose=True)

# Images

Load the true data and the results with GATSBI and NPE

In [12]:
data = np.load(join("plotting_data/camera_samples.npz"))
theta_test,\
obs_test,\
gatsbi_samples,\
npe_samples = data["theta_test"],\
              data["obs_test"],\
              data["gatsbi_samples"],\
              data["npe_samples"]

## Original figure:

In [ ]:
gt_colors = "Greys"
npe_colors = "Greys"
npe_colors_std = "Greys"

def add_cbar(cax, im, lim_min, lim_max):
    cbar = plt.colorbar(im, cax=cax, ticks=[lim_min, lim_max])
    cbar.ax.tick_params(labelsize=40)
    cbar.outline.set_visible(False)
    return cbar

def hide_ax(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

with mpl.rc_context(fname='./matplotlibrc'):

    fig, axes = plt.subplots(6, 12, figsize=(50, 22.5))
    left_adjust = .96

    fig.subplots_adjust(right=.95, wspace=.1)
    caxs = []
    caxs.append(fig.add_axes([left_adjust, .79, .01, .1]))
    caxs.append(fig.add_axes([left_adjust, .655, .01, .1]))
    caxs.append(fig.add_axes([left_adjust, .52, .01, .1]))
    caxs.append(fig.add_axes([left_adjust, .385, .01, .1]))
    caxs.append(fig.add_axes([left_adjust, .25, .01, .1]))
    caxs.append(fig.add_axes([left_adjust, .115, .01, .1]))
    # 135
    # imshow settings
    fontname = "Arial"
    fontsize = 50
    origin = 'lower'
    mn_vmin = 0.
    mn_vmax = 1.

    std_vmin = 0.
    std_vmax = .1

    for i, ax in enumerate(axes.T):
        for j, (samp, tit) in enumerate(zip([theta_test, obs_test, [gatsbi_samples, npe_samples]],
                                            [r"$\theta$", r"$y_{o}$", "Mean"]
                                           )
                                       ):
            # Set ylabel
            if i == 0:
                ax[j].set_ylabel(tit, fontsize=fontsize, fontname=fontname)
                ax[4].set_ylabel("Mean", fontsize=fontsize, fontname=fontname)
                
                ax[3].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                ax[5].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                
            # Plot GT theta and obs
            if j < 2:
                im = ax[j].imshow((samp[i].squeeze()).T[::-1], 
                                  origin=origin,
                                  vmax=mn_vmax, 
                                  vmin=mn_vmin, 
                                  cmap=gt_colors)
                cbar = add_cbar(caxs[j], im, mn_vmin, mn_vmax)
                hide_ax(ax[j])
                
            # Plot NPE / GATSBI samples
            else:
                for k, (ss, mean_vmax, stdev_vmax) in enumerate(zip(samp, [1., 10.], [.1, 1000.])):
#                     mean = np.mean(npe_samples[i], 0).squeeze()
#                     std = np.std(npe_samples[i], 0).squeeze()
                    mean = np.mean(ss[i], 0).squeeze()
                    std = np.std(ss[i], 0).squeeze()
                    im = ax[j+ 2*k].imshow(mean.T[::-1], 
                                        origin=origin,
                                        vmax=mean_vmax,
                                        vmin=mn_vmin, 
                                        cmap=npe_colors)
                    cbar = add_cbar(caxs[j + 2*k], im, mn_vmin, mean_vmax)
                    hide_ax(ax[j + 2*k])

                    im = ax[j + 2*k + 1].imshow(std.T[::-1],
                                                vmax=stdev_vmax,
                                                vmin=std_vmin,
                                                origin=origin,
                                                cmap=npe_colors_std)
                    cbar = add_cbar(caxs[j + 2*k + 1], im, std_vmin, stdev_vmax)
                    hide_ax(ax[j + 2*k + 1])

    fig.text(0.095, .76, 
             "Groundtruth", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")

    fig.text(0.095, .5, 
             "GATSBI", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")
    
    fig.text(0.095, .23, 
             "NPE", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")

    ax = hide_ax(plt.gca())

    # plt.savefig("plots/Figure5.pdf")


## Create posterior samples for the other methods: 

In [ ]:
from glob import glob

In [ ]:
!mkdir plots/camera_model

In [ ]:
root_folder = "/data/localhost/pacchiar/GATSBI_extension_res/"

Define the generator for the task at hand:

In [ ]:
import sys, os
sys.path.append(os.getcwd() + "/../")
from gatsbi.task_utils.camera_model import Generator
from gatsbi.networks import WrapGenMultipleSimulations

gen = Generator()

Transform the data

In [ ]:
obs_test_tensor = torch.from_numpy(obs_test.astype(np.float32))

In [ ]:
obs_test_tensor.shape

In [ ]:
task_name = "camera_model"
df_in = df_10

scoring_rule_list = ["energy_score", "kernel_score"]
patches_list = [None, [5,8], [7,14]]
#scoring_rule_list = ["kernel_score"]
#patches_list = [[5,8]]

samples_list = []

for scoring_rule in scoring_rule_list:
    for patch in patches_list:

        # obtain the identifier
        df = df_in[df_in.scoring_rule == scoring_rule]
        if patch is None: 
            df = df[df.patched_sr == False]
        else:
            df = df[df.patched_sr== True]
            df = df[df.patch_size == patch[1]]         
            
        identifier = df["ID"].item()
        print(identifier)

        # obtain the path
        correct_folder = glob(root_folder + task_name + f"/wandb/*{identifier}*")
        if len(correct_folder) == 0:
            print("No match")
            continue
        correct_folder = correct_folder[0]
        
        # now should load the generative network for that folder:        
        chpt = torch.load(correct_folder +  "/files/checkpoint_models0.pt",map_location=torch.device('cpu') )
        gen.load_state_dict(chpt["generator_state_dict"])

        # wrap the generator so that it can generate multiple samples at once: 
        gen_wrapped = WrapGenMultipleSimulations(gen, n_simulations=100)        
        
        # now feed the observations into the generator to obtain posterior samples: 
        samples_list.append(gen_wrapped(obs_test_tensor).unsqueeze(2).detach().numpy())  # add a dummy dimension to have same 
                

## Plot: best methods wrt GAN
Both Energy and Kernel Score with patch 5_8 perform well. Report those two in the main text
The best method is with kernel_score and patch 5_8

In [ ]:
main_samples = [gatsbi_samples, samples_list[1], samples_list[-2]]

In [ ]:
with mpl.rc_context(fname='./matplotlibrc'):

    fig, axes = plt.subplots(8, 12, figsize=(50, 22.5 * 4 / 3))
    left_adjust = .96

    fig.subplots_adjust(right=.95, wspace=.1)
    caxs = []
    height_caxs = .08
    
    caxs.append(fig.add_axes([left_adjust, .81, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .71, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .61, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .51, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .41, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .31, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .21, .01, height_caxs]))
    caxs.append(fig.add_axes([left_adjust, .11, .01, height_caxs]))

    # 135
    # imshow settings
    fontname = "Arial"
    fontsize = 50
    origin = 'lower'
    mn_vmin = 0.
    mn_vmax = 1.

    std_vmin = 0.
    std_vmax = .1

    #for i, ax in enumerate(axes.T):
    for i, ax in enumerate(axes.T):
        for j, (samp, tit) in enumerate(zip([theta_test, obs_test, main_samples],
                                            [r"$\theta$", r"$y_{o}$", "Mean", "Mean"]
                                           )
                                       ):
            # Set ylabel
            if i == 0:
                ax[j].set_ylabel(tit, fontsize=fontsize, fontname=fontname)
                ax[4].set_ylabel("Mean", fontsize=fontsize, fontname=fontname)
                ax[6].set_ylabel("Mean", fontsize=fontsize, fontname=fontname)
                
                ax[3].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                ax[5].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                ax[7].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                
            # Plot GT theta and obs
            if j < 2:
                im = ax[j].imshow((samp[i].squeeze()).T[::-1], 
                                  origin=origin,
                                  vmax=mn_vmax, 
                                  vmin=mn_vmin, 
                                  cmap=gt_colors)
                cbar = add_cbar(caxs[j], im, mn_vmin, mn_vmax)
                hide_ax(ax[j])
                
            # Plot NPE / GATSBI samples
            else:
                for k, (ss, mean_vmax, stdev_vmax) in enumerate(zip(samp, [1., 1., 1.], [.1, .5, .5])):
#                     mean = np.mean(npe_samples[i], 0).squeeze()
#                     std = np.std(npe_samples[i], 0).squeeze()
                    mean = np.mean(ss[i], 0).squeeze()
                    std = np.std(ss[i], 0).squeeze()
                    #print("method {}, max std {}".format(k,std.max()))
                    #if std.max()>0.4:
                    #    print(k)
                    im = ax[j+ 2*k].imshow(mean.T[::-1], 
                                        origin=origin,
                                        vmax=mean_vmax,
                                        vmin=mn_vmin, 
                                        cmap=npe_colors)
                    cbar = add_cbar(caxs[j + 2*k], im, mn_vmin, mean_vmax)
                    hide_ax(ax[j + 2*k])

                    im = ax[j + 2*k + 1].imshow(std.T[::-1],
                                                vmax=stdev_vmax,
                                                vmin=std_vmin,
                                                origin=origin,
                                                cmap=npe_colors_std)
                    cbar = add_cbar(caxs[j + 2*k + 1], im, std_vmin, stdev_vmax)
                    hide_ax(ax[j + 2*k + 1])

    fig.text(0.095, .8, 
             "Groundtruth", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")

    fig.text(0.095, .6, 
             "GAN", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")
    
    fig.text(0.095, .4, 
             "Energy Score", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")

    fig.text(0.095, .2, 
             "Kernel Score", 
             rotation='vertical', 
             va='center', 
             fontsize=fontsize,
             fontname=fontname,
             fontweight="bold")

    #ax = hide_ax(plt.gca())  # that is useless

    plt.savefig("plots/camera_model/camera_model_best_methods.pdf")

Now create all plots at once

In [ ]:
samples_list_total = [gatsbi_samples] + samples_list

In [ ]:
assert len(samples_list_total) == 7

with mpl.rc_context(fname='./matplotlibrc'):

    fig, axes = plt.subplots((1 + len(samples_list_total)) * 2, 12, figsize=(50, 22.5 * (1 + len(samples_list_total)) / 3))
    left_adjust = .96

    fig.subplots_adjust(right=.95, wspace=.1)

    height_caxs = .04
    start_caxs_pos = 0.11
    factor_caxs_pos = 0.049
    caxs = []
    for i in range(16):
        caxs.append(fig.add_axes([left_adjust, start_caxs_pos + i *factor_caxs_pos, .01, height_caxs]))
    caxs.reverse()
        
    # 135
    # imshow settings
    fontname = "Arial"
    fontsize = 50
    origin = 'lower'
    mn_vmin = 0.
    mn_vmax = 1.

    std_vmin = 0.
    std_vmax = .1

    #for i, ax in enumerate(axes.T):
    for i, ax in enumerate(axes.T):
        for j, (samp, tit) in enumerate(zip([theta_test, obs_test, samples_list_total],
                                            [r"$\theta$", r"$y_{o}$", "Mean"]
                                           )
                                       ):
            # Set ylabel
            if i == 0:
                ax[j].set_ylabel(tit, fontsize=fontsize, fontname=fontname)
                ax[3].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                
                for kk in range(2, 8):
                    ax[2*kk].set_ylabel("Mean", fontsize=fontsize, fontname=fontname)
                    ax[2*kk+1].set_ylabel("SD", fontsize=fontsize, fontname=fontname)
                
            # Plot GT theta and obs
            if j < 2:
                im = ax[j].imshow((samp[i].squeeze()).T[::-1], 
                                  origin=origin,
                                  vmax=mn_vmax, 
                                  vmin=mn_vmin, 
                                  cmap=gt_colors)
                cbar = add_cbar(caxs[j], im, mn_vmin, mn_vmax)
                hide_ax(ax[j])
                
            # Plot NPE / GATSBI samples
            else:
                for k, (ss, mean_vmax, stdev_vmax) in enumerate(zip(samp, [1.] * len(samp), [.1] + [.5]* (len(samp)-1))):
#                     mean = np.mean(npe_samples[i], 0).squeeze()
#                     std = np.std(npe_samples[i], 0).squeeze()
                    mean = np.mean(ss[i], 0).squeeze()
                    std = np.std(ss[i], 0).squeeze()
                    #print("method {}, max std {}".format(k,std.max()))
                    #if std.max()>0.4:
                    #    print(k, std.max())
                    im = ax[j+ 2*k].imshow(mean.T[::-1], 
                                        origin=origin,
                                        vmax=mean_vmax,
                                        vmin=mn_vmin, 
                                        cmap=npe_colors)
                    cbar = add_cbar(caxs[j + 2*k], im, mn_vmin, mean_vmax)
                    hide_ax(ax[j + 2*k])

                    im = ax[j + 2*k + 1].imshow(std.T[::-1],
                                                vmax=stdev_vmax,
                                                vmin=std_vmin,
                                                origin=origin,
                                                cmap=npe_colors_std)
                    cbar = add_cbar(caxs[j + 2*k + 1], im, std_vmin, stdev_vmax)
                    hide_ax(ax[j + 2*k + 1])

                    
    names = ["Groundtruth", "GAN", "Energy", "Energy 5,8", "Energy 7,14", "Kernel", "Kernel 5,8", "Kernel 7,14"]
    positions = [.835,.74,.65, .55, .45, .35, .25625, .15]
    for pos, name in zip(positions, names):
        fig.text(0.095, pos, 
                 name, 
                 rotation='vertical', 
                 va='center', 
                 fontsize=fontsize,
                 fontname=fontname,
                 fontweight="bold")

    plt.savefig("plots/camera_model/camera_model_all_methods.pdf")